In [1]:
import cv2
#import pandas as pd

import numpy as np
# 2cm x 2cm


In [2]:
not_detect_imgs = []
corners_dict = {}
for idx in range(1,26): # testing first two imgs
    img = cv2.imread(f"img{idx}.jpg")
    img = np.uint8(img)
    gray_scaled_img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

    ans, corners= cv2.findChessboardCorners(gray_scaled_img, patternSize= (9,6))
    if ans is False:
        not_detect_imgs.append(idx)
    else:
        corners_dict[idx] = corners
    #print(len(corners))
    
    
    #print(corners)
    # x = corners[0][0]
    # y = corners[0][1]
    # print(f"x: {x}, y: {y}")
    # print("ans", ans)
    # print('corner:', corners)
    # print('------------')

# print(not_detect_imgs)
# print(corners_dict)

print(not_detect_imgs)

[1, 4, 5, 10, 12, 13, 19, 20]


In [3]:

for corners in corners_dict.values():
    for coords in corners: #2621. 2016.5
        #print(i[0])
        x = coords[0][0]
        y = coords[0][1]
        print(x,y)
        center = (int(x), int(y))
        print(center)
        break

544.98474 316.937
(544, 316)
610.429 336.40582
(610, 336)
581.5 405.5
(581, 405)
602.76904 356.7643
(602, 356)
627.68677 378.15363
(627, 378)
608.0 427.0
(608, 427)
601.5 171.5
(601, 171)
551.5229 238.32903
(551, 238)
558.09174 157.32755
(558, 157)
704.82367 227.25104
(704, 227)
703.4904 299.05634
(703, 299)
610.579 289.07443
(610, 289)
551.0438 374.34348
(551, 374)
555.0885 355.52173
(555, 355)
519.81104 392.5449
(519, 392)
637.2707 403.75052
(637, 403)
642.0843 278.60364
(642, 278)


In [4]:
for i in corners_dict.values(): # checking amount of corners in img2
    print(len(i))

54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54
54


In [12]:

square_size = 0.02 #2 cm
obj_pts = np.zeros((9*6, 3), np.float32)
obj_pts[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) * square_size


all_obj_pts = []
all_img_pts = []

auto_only_imgs = []

manual_only_imgs = []

def click_event(event, x, y, flags, param):
    current_img = param['img']
    current_name = param['name']
    if event == cv2.EVENT_LBUTTONDBLCLK:
        print(x,y) # coordinate
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(current_img, f'{x}, {y}', (x,y), font, 1, (255,0,0), 2)
        cv2.imshow(window_name, current_img)
        param['point'].append([x,y])
    
    if event == cv2.EVENT_RBUTTONDBLCLK:
        print(x,y)
        font = cv2.FONT_HERSHEY_SIMPLEX
        b,g,r = current_img[y,x] 
        cv2.putText(img,f'{b}, {g}, {r}', (x,y), font, 1, (255, 255,0), 2)
        cv2.imshow(window_name, current_img)
        #points += [x,y]

    print(f"Manual Points: {param['point']}")



# linear interpolation
def linear_interpolation(pts, pattern_size):
    tl, tr, br, bl = np.array(pts[0],dtype = 'float32'),  np.array(pts[1],dtype = 'float32'), np.array(pts[2],dtype = 'float32'), np.array(pts[3],dtype = 'float32')
    #print(type(tl))
    cols, rows = pattern_size
    print(cols)
    grid = []
    for row in range(rows):
        ver_int = row / (rows - 1)
        row_start = (1 - ver_int) * tl + ver_int * bl
        row_end = (1 - ver_int)*  tr + ver_int * br
        
        for col in range(cols):
            hor_int = col / (cols -1)
            point = (1 - hor_int) * row_start + hor_int * row_end
            grid.append(point)
    return np.array(grid, dtype= 'float32').reshape(-1, 1, 2)

pattern_size = (9,6)

window_name = 'chessboard_tool'
all_result = {}
# read input image
for idx in range(1,26):

    img = cv2.imread(f'img{idx}.jpg')

# convert the input image to a grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    print(corners)
    # if chessboard corners are detected
    if ret == True:
    
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners,ret)
        all_img_pts.append(corners)
        auto_only_imgs.append(corners)
        all_obj_pts.append(obj_pts)

        cv2.imshow(window_name,img)
        cv2.waitKey(0)

    elif ret == False:
        # clicking (TL -> TR -> BR -> BL)
        manual_pts = []
        cv2.imshow(window_name, img)
        cv2.setMouseCallback(window_name, click_event, param= {'img': img, 'name': f'Image{idx},', 'point':manual_pts})
        cv2.waitKey(0)

        if len(manual_pts) == 4:
            interpolated_corners = linear_interpolation(manual_pts, (9,6))
            all_img_pts.append(interpolated_corners)
            manual_only_imgs.append(interpolated_corners)
            all_obj_pts.append(obj_pts)

            img_vis = img.copy()
            cv2.drawChessboardCorners(img_vis, (9,6), interpolated_corners, True)
            cv2.imshow(window_name, img_vis)
            print("Showing interpolated grid. Press any key to continue to next image.")
            cv2.waitKey(0) # Wait again so you can actually see the result
        
        else:
            print(f"Warning: Only {len(manual_pts)} points clicked. Skipping image {idx}.")
    
cv2.destroyWindow(window_name)



None
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manual Points: []
Manua

In [ ]:
#print(manual_only_imgs)

def calibrate(object_list, img_list, gray_img):

    ret, matrix, distort, rvecs, tvec = cv2.calibrateCamera(object_list, img_list, gray_img, None, None, flags = 0)

    return ret, matrix, distort, rvecs, tvec

#Run 1: All Images

ret1, matrix1, distort1, rvecs1, tvec1 = calibrate(all_obj_pts, all_img_pts, gray.shape[::-1] )

#Run 2: Only Five images each
run2_imgs = auto_only_imgs[:5] + manual_only_imgs[:5]

object_10 = [obj_pts]*10

ret2, matrix2, distort2, rvecs2, tvec2 = calibrate(object_10, run2_imgs, gray.shape[::-1] )

#Run 3: Automatic 5 images

run3_imgs = auto_only_imgs[:5] 

object_5_3 = [obj_pts]*5

ret3, matrix3, distort3, rvecs3, tvec3 = calibrate(object_5_3, run3_imgs, gray.shape[::-1] )


#rt_total = [(rvecs1, tvec1, 'Run 1'), (rvecs2, tvec2, 'Run2'), (rvecs3, tvec3, 'Run 3')]
runs = [(matrix1, distort1, "Run 1"), (matrix2, distort2, "Run 2"), (matrix3, distort3, "Run 3")]

In [64]:
axis = np.float32([[0.06, 0 ,0], [0, 0.06, 0], [0, 0,-0.06]]).reshape(-1,3)
#print(rt_total)
#cube: 8 corners

s = 0.04

cube_points = np.float32([[0,0,0], [s,0,0], [s,s,0], [0,s,0], [0,0,-s], [s,0,-s], [s,s,-s], [0,s,-s]])

for matrix, distort, label in runs:
    test_img = cv2.imread('test.jpg')
    test_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    
    ret_test, test_corners = cv2.findChessboardCorners(test_gray, (9,6), None)
    origin_pixel = tuple(test_corners[0].ravel().astype(int))

    ret, rvec, tvec= cv2.solvePnP(obj_pts, test_corners, matrix, distort)

    #We want to project the axis onto image
    imgpoints_axis, _ = cv2.projectPoints(axis, rvec, tvec, matrix, distort)

    imgpoints_cube, _ = cv2.projectPoints(cube_points, rvec, tvec, matrix, distort)
    #print('imgpoints_cube:', imgpoints_cube)
    # Convert your projected points to integers for drawing
    axis_pix = imgpoints_axis.reshape(-1, 2).astype(int)
    cube_pix = imgpoints_cube.reshape(-1, 2).astype(int)
    #print('cube_pix:', cube_pix)


    # create polygon
    #chaning color of polygon based on orientation and position
    ori_var, pos_var = rvec, tvec
    rotate_intensity = np.linalg.norm(ori_var) #calculate the norm of a matrix
    ori_color = int(min(255, rotate_intensity*50)) # the ori_color can range from 0 to 255

    trans_distance = np.linalg.norm(pos_var)
    trans_color = int(min(255, trans_distance* 50))
    
    top_plane_pts = cube_pix[4:8]
    #print(top_plane_pts)
    cv2.fillConvexPoly(img = test_img, points= top_plane_pts, color = (trans_color, ori_color,  255))
    top_center_3d = np.float32([[s/2, s/2, -s]]) #3d coord
    center_pt,_ = cv2.projectPoints(top_center_3d, rvec, tvec, matrix, distort)
    center_pixel = tuple(center_pt.reshape(-1,2)[0].ravel().astype(int))
    print(f'center_pixel: {center_pixel}')
    print(f'top center point of the polygon is: {center_pt}')
    cv2.circle(test_img, center_pixel, 5, (0,0,255), 1)
    #distance
    R, _ = cv2.Rodrigues(rvec) #convert rvec into rMatrix return(Rotation_matrix, Jaccobian)
    cam_pt = np.dot(R, top_center_3d.T)+  tvec
    cam_distance = np.linalg.norm(cam_pt)
    print(f'cam_pt:{cam_pt}')
    print(f'cam_distance: {cam_distance}')


    
    # Draw 3 axies
    cv2.line(test_img, origin_pixel, tuple(axis_pix[0]), (0, 0, 255 ), 5)
    cv2.line(test_img, origin_pixel, tuple(axis_pix[1]), (0, 255, 0), 5)
    cv2.line(test_img, origin_pixel, tuple(axis_pix[2]), (255, 0, 0), 5)
    
    #  pillars of the cube
    cv2.line(test_img, tuple(cube_pix[0]), tuple(cube_pix[4]), (0, 0, 0), 2)
    cv2.line(test_img, tuple(cube_pix[1]), tuple(cube_pix[5]), (0, 0, 0), 2)
    cv2.line(test_img, tuple(cube_pix[2]), tuple(cube_pix[6]), (0, 0, 0), 2)
    cv2.line(test_img, tuple(cube_pix[3]), tuple(cube_pix[7]), (0, 0, 0), 2)

    #connect the corners
    cv2.polylines(test_img, [cube_pix[0:4]], True, (0, 0, 0), 2)
    cv2.polylines(test_img, [cube_pix[4:8]], True, (0, 0, 0), 2)

    
    cv2.circle(test_img, center_pixel, 5, (0, 0, 255), -1) 
    cv2.putText(test_img, f"{cam_distance:.2f}m", center_pixel, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)




    cv2.imshow(f'chessboard_window: {label}', test_img)
    cv2.waitKey(0)
    cv2.destroyWindow(f'chessboard_window: {label}')

center_pixel: (599, 498)
top center point of the polygon is: [[[599.86975 498.9313 ]]]
cam_pt:[[-0.02435671]
 [ 0.03776107]
 [ 1.49367715]]
cam_distance: 1.49435289265669
center_pixel: (597, 516)
top center point of the polygon is: [[[597.2277  516.17316]]]
cam_pt:[[-0.00997206]
 [ 0.04850818]
 [ 4.10768009]]
cam_distance: 4.10797860518652
center_pixel: (607, 445)
top center point of the polygon is: [[[607.5585  445.50507]]]
cam_pt:[[-0.02461136]
 [ 0.02494992]
 [ 0.48104059]]
cam_distance: 0.4823155287281278


Draw a polygon that covers the top side of the cube (check the fillConvexPoly function). Place a dot in the center of the top plane and add a text that shows the distance of this point to the camera, in meters. The color of the top side polygon should change with the postion and orientation of the center of the top plane, relative to the camera. Use only 3D-coordinates and extrinsics.

In [ ]:
# draw a polygon

cv2.fillConvexPoly()